In [1]:
import pandas as pd
import numpy as np
from typing import Dict

from PIL import Image
from datasets import load_dataset, Dataset, Features, Value

from sklearn.model_selection import train_test_split

# Data

In [76]:
df = pd.read_json('../data/vlm.jsonl', lines=True)
# df = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=1000, random_state=1)
# df = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=3000, random_state=1)

# obj_lst = pd.read_json('../data/categories.jsonl', lines=True)
# categories = np.unique(obj_lst.to_numpy())
# # print(categories)

# id2label = {index: x for index, x in enumerate(categories, start=0)}
# label2id = {v: k for k, v in id2label.items()}

categories = list(filter(lambda x: x != None, pd.DataFrame([i for i in list(df['annotations'].apply(lambda x: [i['caption'] for i in x]))]).to_numpy().flatten()))
num_cat = len(categories)
categories = pd.DataFrame(categories)
categories.drop_duplicates(inplace=True)
categories = categories.values.flatten().tolist()
id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

# label2id

In [93]:
df['annotations'].apply(lambda x: [cap['caption'] for cap in x])

0       [grey missile, red, white, and blue light airc...
1       [grey camouflage fighter jet, grey and white f...
2       [grey drone, white and black drone, yellow, re...
3       [white and black light aircraft, grey and blac...
4       [white, red, and green fighter plane, black ca...
                              ...                        
5102    [blue and red light aircraft, white fighter je...
5103    [white and blue cargo aircraft, black and brow...
5104    [yellow helicopter, red, white, and blue light...
5105    [white fighter plane, red fighter plane, white...
5106    [white cargo aircraft, red and black drone, gr...
Name: annotations, Length: 5107, dtype: object

In [110]:
df = pd.read_json('../data/vlm.jsonl', lines=True)
df["image_id"] = df.index

df['width'] = df['image'].apply(lambda x: Image.open('../data/images/' + x).size)
df['height'] = df['width'].apply(lambda x: x[1])
df['width'] = df['width'].apply(lambda x: x[0])

# df['img'] = df['image'].apply(lambda x: Image.open('../data/images/' +x))

# df['area'] = df['annotations'].apply(lambda x: {"area":[i['bbox'][2] * i['bbox'][3] for i in x]})
# df['bbox'] = df['annotations'].apply(lambda x: {"bbox":[i['bbox'] for i in x]})

# def check_str(caption, categories, label2id):
# 	for cat in categories:
# 		if cat in caption:
# 			return label2id[cat]

# df['category'] = df['annotations'].apply(lambda x: {'category':[check_str(i['caption'], categories, label2id) for i in x]})

id_len = 0
def id_label(x):
	global id_len
	res = [i for i in range(id_len, id_len + len(x))]
	id_len += len(x)
	return res
def wrap_data(x):
	dct = {"id":id_label(x), 
		"area":[i['bbox'][2] * i['bbox'][3] for i in x],
		"bbox":[i['bbox'] for i in x],
		"category":[i for i in range(len(x))], 
		"caption":[i['caption'] for i in x]
	}
	return dct


df['objects'] = df['annotations'].apply(lambda x: wrap_data(x))
df = df[['image_id', 'image', 'width', 'height', 'objects']]
df

,image_id,image,width,height,objects
0,0,image_0.jpg,1520,870,"{'id': [0, 1, 2, 3], 'area': [7296, 672, 4864,..."
1,1,image_1.jpg,1520,870,"{'id': [4, 5, 6, 7, 8, 9], 'area': [2304, 6912..."
2,2,image_2.jpg,1520,870,"{'id': [10, 11, 12, 13, 14, 15], 'area': [2912..."
3,3,image_3.jpg,1520,870,"{'id': [16, 17, 18, 19, 20, 21, 22], 'area': [..."
4,4,image_4.jpg,1520,870,"{'id': [23, 24, 25], 'area': [3360, 1152, 3648..."
...,...,...,...,...,...
5102,5102,image_5102.jpg,1520,870,"{'id': [27886, 27887, 27888, 27889, 27890, 278..."
5103,5103,image_5103.jpg,1520,870,"{'id': [27894, 27895, 27896, 27897, 27898, 278..."
5104,5104,image_5104.jpg,1520,870,"{'id': [27901, 27902, 27903, 27904, 27905], 'a..."
5105,5105,image_5105.jpg,1520,870,"{'id': [27906, 27907, 27908], 'area': [1584, 2..."


In [102]:
df['objects'][0]

{'id': [0, 1, 2, 3],
 'area': [7296, 672, 4864, 5376],
 'bbox': [[912, 164, 48, 152],
  [1032, 80, 24, 28],
  [704, 508, 76, 64],
  [524, 116, 112, 48]],
 'category': [0, 1, 2, 3],
 'caption': ['grey missile',
  'red, white, and blue light aircraft',
  'green and black missile',
  'white and red helicopter']}

In [4]:
df = df.drop(columns=['annotations'])

counter = 0
lst = []
for i in df['category']:
	size = len(i['category'])
	each = list(np.arange(counter, counter + size))
	lst.append({'id':each})
	counter += size
df['id'] = lst

df_vlm = pd.DataFrame({
	"file_name" : df['image'],
	"image_id" : df['image_id'],
	"image" : df['img'],
	"width" : df['width'],
	"height": df['height'],
	"objects": df.apply(lambda x: {'id': x['id']['id'], 'area': x['area']['area'],'bbox': x['bbox']['bbox'], 'category': x['category']['category']}, axis=1)
}).sort_index(axis=0)
# df_vlm.to_csv('meta_data.csv', index=False)
df_vlm

,file_name,image_id,image,width,height,objects
5074,image_5074.jpg,5074,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [0, 1, 2, 3, 4, 5, 6, 7], 'area': [2640..."
2711,image_2711.jpg,2711,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [8, 9, 10, 11, 12], 'area': [2048, 2496..."
3670,image_3670.jpg,3670,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [13, 14, 15, 16], 'area': [960, 3072, 1..."
2377,image_2377.jpg,2377,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [17, 18, 19, 20, 21, 22, 23], 'area': [..."
4195,image_4195.jpg,4195,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [24, 25, 26, 27, 28, 29, 30], 'area': [..."
...,...,...,...,...,...,...
1357,image_1357.jpg,1357,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16302, 16303, 16304, 16305, 16306, 163..."
3892,image_3892.jpg,3892,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16308, 16309, 16310, 16311, 16312, 163..."
3258,image_3258.jpg,3258,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16314, 16315, 16316, 16317, 16318], 'a..."
3507,image_3507.jpg,3507,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16319, 16320, 16321], 'area': [1536, 1..."


# Split Dataset

In [5]:
df_vlm_csv = df_vlm.drop(columns=['image'])
df_vlm_csv.to_csv('../data/images/metadata.csv', index=False)
# pd.DataFrame({
# 	'name':df_vlm['file_name'],
# 	'img':df_vlm['image']
# }).apply(lambda x: x[1].save('../vlm-images2/images/' + x[0]), axis=1)